In [22]:
import pandas as pd
import os

In [23]:
file_path = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies"
print(file_path)

C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies


In [24]:
list_of_files = os.listdir(file_path)
path_dir = []
for path in list_of_files:
    new_path = f"{file_path}\\{path}"
    path_dir.append(new_path)


In [41]:
import os
import pandas as pd
import json

for file_dir in path_dir:
    company_list_dir = os.listdir(file_dir)
    
    for company in company_list_dir:
        company_path = os.path.join(file_dir, company)
        
        # Skip if it's not a directory
        if not os.path.isdir(company_path):
            continue  

        new_file_path = os.path.join(company_path, "Pruned_Excel")
        
        # Skip if "Pruned_Excel" folder doesn't exist
        if not os.path.exists(new_file_path):
            print(f"⚠️ Skipping: {new_file_path} (Not Found)")
            continue

        # Initialize dictionary for storing parameters
        company_params = {
            "Balance-sheet": [],
            "Cash-flow": [],
            "Profit-loss": [],
            "Quarterly-resul": [],
            "Ratios": [],
            "Year": []
        }

        # Get list of Excel files
        prunedExcel_file_list = [
            file for file in os.listdir(new_file_path)
            if file.endswith(".xlsx") or file.endswith(".xls")
        ]
        
        for excel_file in prunedExcel_file_list:
            excel_file_path = os.path.join(new_file_path, excel_file)
            df = pd.read_excel(excel_file_path)

            parameters = df.columns[1:].tolist()  # Extract column names excluding the first column

            if "Balance-sheet" in excel_file:
                company_params["Balance-sheet"].extend(sorted(parameters))
            elif "Cash-flow" in excel_file:
                company_params["Cash-flow"].extend(sorted(parameters))
            elif "Profit-loss" in excel_file:
                company_params["Profit-loss"].extend(sorted(parameters))
            elif "Quarterly-resul" in excel_file:
                company_params["Quarterly-resul"].extend(sorted(parameters))
            elif "Ratios" in excel_file:
                company_params["Ratios"].extend(sorted(parameters))
            elif "Year" in excel_file:
                company_params["Year"].extend(sorted(parameters))
        
        # Save the JSON file inside the respective company directory
        json_file_path = os.path.join(company_path, "company_params.json")
        with open(json_file_path, "w") as f:
            json.dump(company_params, f, indent=4)

        print(f"✅ JSON file saved at: {json_file_path}")


✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Ahluwalia Contracts India Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Ajmera Realty and Infra India Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Alpine Housing Dev Corp Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\AMJ Land Ltd\company_params.json
✅ JSON file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tu

In [ ]:
# working

import os
import json
import pandas as pd
from collections import Counter

# Define the root directory where all sectors are stored
root_dir = r"C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies"

# Get all sector directories
sector_dirs = [os.path.join(root_dir, sector) for sector in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, sector))]

# Categories to process
categories = ["Balance-sheet", "Cash-flow", "Profit-loss", "Quarterly-resul", "Ratios", "Year"]

for sector_path in sector_dirs:
    sector_name = sector_path.split("\\")[-1]
    list_of_company_params = []

    # Get all company directories inside the sector
    company_list_dir = os.listdir(sector_path)

    for company in company_list_dir:
        company_path = os.path.join(sector_path, company)
        json_file_path = os.path.join(company_path, "company_params.json")

        # Skip if not a directory or company_params.json doesn't exist
        if not os.path.isdir(company_path) or not os.path.exists(json_file_path):
            continue  

        # Read the JSON file
        with open(json_file_path, "r") as f:
            company_params = json.load(f)

        list_of_company_params.append(company_params)

    # Save the aggregated data in company_params.json inside the sector folder
    aggregated_file_path = os.path.join(sector_path, "company_params.json")
    with open(aggregated_file_path, "w") as f:
        json.dump(list_of_company_params, f, indent=4)
    
    print(f"Aggregated JSON saved at: {aggregated_file_path}")

    # Dictionary to store sorted parameter counts for each category
    sorted_data = {}
    excel_data = {}  # Dictionary to store data for Excel export

    for category in categories:
        # Extract lists for the current category
        category_lists = [datas.get(category, []) for datas in list_of_company_params]
        
        # Flatten and count occurrences of all parameters in the category
        all_parameters = [param for sublist in category_lists for param in sublist]
        param_counts = Counter(all_parameters)
        
        # Sort parameters by count in descending order
        sorted_param_counts = dict(sorted(param_counts.items(), key=lambda item: item[1], reverse=True))
        
        # Store in the main dictionary with category name as the key
        sorted_data[category] = sorted_param_counts

        # Prepare data for Excel (convert dictionary to list of tuples)
        excel_data[category] = [(param, count) for param, count in sorted_param_counts.items()]

    # Save sorted JSON inside the same sector directory
    sorted_file_path = os.path.join(sector_path, f"{sector_name}.json")
    with open(sorted_file_path, "w") as f:
        json.dump(sorted_data, f, indent=4)

    print(f"Sorted JSON saved at: {sorted_file_path}")

    # Convert the sorted data to an Excel file
    excel_file_path = os.path.join(sector_path, f"{sector_name}.xlsx")
    with pd.ExcelWriter(excel_file_path) as writer:
        for category, data in excel_data.items():
            df = pd.DataFrame(data, columns=["Parameter", "Count"])
            df.to_excel(writer, sheet_name=category, index=False)

    print(f"Sorted Excel file saved at: {excel_file_path}")


Aggregated JSON saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\company_params.json
Sorted JSON saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Construction - Residential & Commercial Complexes.json
Sorted Excel file saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Construction - Residential & Commercial Complexes\Construction - Residential & Commercial Complexes.xlsx
Aggregated JSON saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Companies\Hospital & Healthcare Services\company_params.json
Sorted JSON saved at: C:\Users\sharm\OneDrive\Desktop\Kishan\Contractzy\WebScrapping\Tutorial\Financial_Data\MoneyControl\Compa